### ORI Final Analysis
### 03. Natural and Cultural Resources

#### 10/24/2024
#### This script processes data and creates the natioanl security submodel. 

In [1]:
#import system modules and set environment
import arcpy
import os
from arcpy import env
from arcpy.sa import *
arcpy.env.overwriteOutput = True
arcpy.env.workspace = 'C:/Users/Eliza.Carter/Documents/Projects/California/ORI'

#define spatial reference (EPSG for NAD 83 UTM Zone 11N)
spatial_ref = arcpy.SpatialReference(26911)

#create a copy of the ORI area hex grid
#hex_grid = 'data/ORI_Area_of_Interest_grid.shp'
hex_grid = 'data/input_data/constrained.shp'
nc_submodel = 'data/input_data/natural_cultural_submodel'
arcpy.management.CopyFeatures(hex_grid, nc_submodel)

#define area of interest
aoi = 'data/ORI_Area_of_Interest.shp'

#create geopackage
nc_gpkg_path = 'data/input_data/ORI_Natural_Cultural.gpkg'
if not os.path.exists(nc_gpkg_path):
    arcpy.management.CreateSQLiteDatabase(nc_gpkg_path, "GEOPACKAGE")

In [2]:
# Humpback Whale Critical Habitat (ori_nc_01)

#define file
file_path = 'data/source_data/WhaleHumpback_MexicoDPS_20210421.shp'

#create feature layer for the file
arcpy.management.MakeFeatureLayer(file_path, "critical_hab_intersect")

#select file where it itersects the grid
arcpy.management.SelectLayerByLocation(
    in_layer= "critical_hab_intersect",
    overlap_type="INTERSECT",
    select_features= nc_submodel,
    search_distance=None,
    selection_type="NEW_SELECTION",
    invert_spatial_relationship="NOT_INVERT"
)

#project to defined spatial reference
file_project = 'in_memory/file_projected'
arcpy.management.Project("critical_hab_intersect", file_project, spatial_ref)

#save to geopackage
output_name = 'Humpback_Whale_Critical_Habitat'
arcpy.conversion.FeatureClassToFeatureClass(file_project, nc_gpkg_path, output_name)

#identify overlap between this layer and the hex grid; score overlapping cells using predetermined value
file_selection = arcpy.management.SelectLayerByLocation(nc_submodel,
                                                                     'INTERSECT',
                                                                     file_project)
arcpy.management.CalculateField(file_selection, 'ori_nc_01', '0.2', field_type = "DOUBLE")

file_selection = arcpy.management.SelectLayerByLocation(nc_submodel, 
                                                        'INTERSECT', 
                                                        file_project, 
                                                        invert_spatial_relationship=True)
arcpy.management.CalculateField(file_selection, 'ori_nc_01', '1', field_type="DOUBLE")

<Result 'natural_cultural_submodel_La2'>

In [3]:
# Natural and Cultural Resources Submodel

#call shapefile
nc_submodel = 'data/input_data/natural_cultural_submodel.shp'

#create a new field for national security
nc_field = "nc_final"

existing_fields = [f.name for f in arcpy.ListFields(nc_submodel)]

if nc_field not in existing_fields:
    arcpy.management.AddField(nc_submodel, nc_field, 'DOUBLE')
    
#expression to get final constraints
final_expression = '(!ori_nc_01!)'

# Calculate the geometric mean and store the result in the 'final_nc' attribute
arcpy.CalculateField_management(nc_submodel, nc_field, final_expression, 'PYTHON3')


<Result 'C:/Users/Eliza.Carter/Documents/Projects/California/ORI\\data/input_data/natural_cultural_submodel.shp'>